## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config_Copy1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iralaya,HN,15.0000,-83.2333,79.32,86,94,12.12,overcast clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,84.36,74,96,15.05,overcast clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,67.26,52,0,10.02,clear sky
3,3,Krasnoselkup,RU,65.7000,82.4667,20.75,73,80,5.03,broken clouds
4,4,Kapaa,US,22.0752,-159.3190,82.38,79,100,16.11,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature that you would like for your trip? "))
max_temp = float(input("What is the maximum temperature that you would like for your trip? "))

What is the minimum temperature that you would like for your trip? 60
What is the maximum temperature that you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Port Alfred,ZA,-33.5906,26.8910,67.26,52,0,10.02,clear sky
10,10,Stephenville,US,32.2207,-98.2023,67.15,63,0,3.44,clear sky
12,12,Port Elizabeth,ZA,-33.9180,25.5701,62.31,95,75,16.11,broken clouds
13,13,Guerrero Negro,MX,27.9769,-114.0611,60.51,89,15,15.55,few clouds
25,25,Busselton,AU,-33.6500,115.3333,63.25,48,0,13.15,clear sky
28,28,Ribeira Grande,PT,38.5167,-28.7000,63.30,88,100,29.80,light rain
35,35,Oytal,KZ,42.9057,73.2697,64.76,62,48,5.53,scattered clouds
36,36,Dunedin,NZ,-45.8742,170.5036,60.80,68,100,3.00,overcast clouds
44,44,Vostok,RU,46.4856,135.8833,60.42,55,100,15.50,light rain
57,57,Esperance,AU,-33.8667,121.9000,63.07,52,14,8.08,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             110
City                   110
Country                107
Lat                    110
Lng                    110
Max Temp               110
Humidity               110
Cloudiness             110
Wind Speed             110
Current Description    110
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

Unnamed: 0             107
City                   107
Country                107
Lat                    107
Lng                    107
Max Temp               107
Humidity               107
Cloudiness             107
Wind Speed             107
Current Description    107
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,67.26,clear sky,-33.5906,26.8910,
10,Stephenville,US,67.15,clear sky,32.2207,-98.2023,
12,Port Elizabeth,ZA,62.31,broken clouds,-33.9180,25.5701,
13,Guerrero Negro,MX,60.51,few clouds,27.9769,-114.0611,
25,Busselton,AU,63.25,clear sky,-33.6500,115.3333,
28,Ribeira Grande,PT,63.30,light rain,38.5167,-28.7000,
35,Oytal,KZ,64.76,scattered clouds,42.9057,73.2697,
36,Dunedin,NZ,60.80,overcast clouds,-45.8742,170.5036,
44,Vostok,RU,60.42,light rain,46.4856,135.8833,
57,Esperance,AU,63.07,few clouds,-33.8667,121.9000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,67.26,clear sky,-33.5906,26.8910,The Halyards Hotel
10,Stephenville,US,67.15,clear sky,32.2207,-98.2023,La Quinta Inn & Suites by Wyndham Stephenville
12,Port Elizabeth,ZA,62.31,broken clouds,-33.9180,25.5701,39 On Nile Guest House
13,Guerrero Negro,MX,60.51,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
25,Busselton,AU,63.25,clear sky,-33.6500,115.3333,Observatory Guest House


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   107
Country                107
Max Temp               107
Current Description    107
Lat                    107
Lng                    107
Hotel Name             107
dtype: int64

In [10]:
# 8a. Create the output File (CSV)


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat","Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0,31.0),zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))